In [1]:
!pip install pycoingecko --quiet


In [2]:
import pandas as pd
import os

DATA_DIR = "/Users/harshit/Downloads/Research-Commons-Quant/okx-midcap-token-index/dataframes"
token_data = {}

for file in os.listdir(DATA_DIR):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(DATA_DIR, file))
        token = file.replace(".csv", "")
        token_data[token] = df


In [3]:
selected_tokens = []

for token, df in token_data.items():
    df['liquidity'] = df['close'] * df['volume']
    df['volume_pass'] = df['volume'] > 1_000_000
    df['liquidity_pass'] = df['liquidity'] > 700_000

    vol_coverage = df['volume_pass'].mean()
    liq_coverage = df['liquidity_pass'].mean()

    if vol_coverage >= 0.8 and liq_coverage >= 0.8:
        selected_tokens.append(token)

print(f"🎯 Tokens that pass both filters: {len(selected_tokens)}")
print(selected_tokens)


🎯 Tokens that pass both filters: 55
['PYTH_USDT', 'CSPR_USDT', 'TON_USDT', 'NEAR_USDT', 'CELO_USDT', 'CRV_USDT', 'HBAR_USDT', 'FET_USDT', 'FIL_USDT', 'ADA_USDT', 'STX_USDT', 'WLD_USDT', 'AEVO_USDT', 'ARB_USDT', 'SATS_USDT', 'YGG_USDT', 'SAND_USDT', 'MEW_USDT', 'CORE_USDT', 'BIGTIME_USDT', 'WIF_USDT', 'TRX_USDT', 'SUSHI_USDT', 'VELO_USDT', 'MERL_USDT', 'ZK_USDT', 'XRP_USDT', 'ATH_USDT', 'NOT_USDT', 'SUI_USDT', 'ZETA_USDT', 'ETHFI_USDT', 'PEPE_USDT', 'OP_USDT', 'PRCL_USDT', 'BONK_USDT', 'PEOPLE_USDT', 'DOGE_USDT', 'USDC_USDT', 'MEME_USDT', 'AIDOGE_USDT', 'CFX_USDT', 'JUP_USDT', 'APE_USDT', 'ZBCN_USDT', 'SHIB_USDT', 'CETUS_USDT', 'XLM_USDT', 'DYDX_USDT', 'TURBO_USDT', 'GALA_USDT', 'FLOKI_USDT', 'W_USDT', 'LDO_USDT', 'STRK_USDT']


In [4]:
from pycoingecko import CoinGeckoAPI
import pandas as pd

cg = CoinGeckoAPI()

# Your token CSVs already loaded and filtered for vol/liquidity
selected_tokens = ['ZRX_USDT', 'ZIL_USDT', 'WAXP_USDT', 'UMA_USDT', 'ZRO_USDT']  # Example

# Helper: Get mapping from symbol to CoinGecko ID
def build_symbol_to_id_map():
    coins = cg.get_coins_list()
    return {coin['symbol'].upper(): coin['id'] for coin in coins}

symbol_to_id = build_symbol_to_id_map()

# Final tokens that pass all filters
mcap_filtered = []

for token in selected_tokens:
    symbol = token.split('_')[0]  # "ZRX_USDT" -> "ZRX"
    coin_id = symbol_to_id.get(symbol.upper())

    if not coin_id:
        print(f"⚠️ No CoinGecko ID for {symbol}")
        continue

    try:
        data = cg.get_coin_by_id(coin_id)
        mcap = data['market_data']['market_cap']['usd']
        print(f"{symbol}: ${mcap:,.2f}")

        if 50_000_000 <= mcap <= 150_000_000:
            mcap_filtered.append(token)
    except Exception as e:
        print(f"❌ Error fetching mcap for {symbol}: {e}")

print(f"\n🎯 Tokens with market cap between $50M and $150M: {len(mcap_filtered)}")
print(mcap_filtered)


ZRX: $199,440,998.00
ZIL: $221,001,049.00
WAXP: $72,466,202.00
UMA: $97,777,125.00
ZRO: $215,480,805.00

🎯 Tokens with market cap between $50M and $150M: 2
['WAXP_USDT', 'UMA_USDT']


In [6]:
from pycoingecko import CoinGeckoAPI
from datetime import datetime
import time

# 1. Init
cg = CoinGeckoAPI()
date_str = "01-08-2024"

# 2. Tokens you're analyzing
tokens_to_check = [
    'PYTH_USDT', 'CSPR_USDT', 'TON_USDT', 'NEAR_USDT', 'CELO_USDT', 'CRV_USDT', 'HBAR_USDT', 'FET_USDT',
    'FIL_USDT', 'ADA_USDT', 'STX_USDT', 'WLD_USDT', 'AEVO_USDT', 'ARB_USDT', 'SATS_USDT', 'YGG_USDT',
    'SAND_USDT', 'MEW_USDT', 'CORE_USDT', 'BIGTIME_USDT', 'WIF_USDT', 'TRX_USDT', 'SUSHI_USDT', 'VELO_USDT',
    'MERL_USDT', 'ZK_USDT', 'XRP_USDT', 'ATH_USDT', 'NOT_USDT', 'SUI_USDT', 'ZETA_USDT', 'ETHFI_USDT',
    'PEPE_USDT', 'OP_USDT', 'PRCL_USDT', 'BONK_USDT', 'PEOPLE_USDT', 'DOGE_USDT', 'USDC_USDT', 'MEME_USDT',
    'AIDOGE_USDT', 'CFX_USDT', 'JUP_USDT', 'APE_USDT', 'ZBCN_USDT', 'SHIB_USDT', 'CETUS_USDT', 'XLM_USDT',
    'DYDX_USDT', 'TURBO_USDT', 'GALA_USDT', 'FLOKI_USDT', 'W_USDT', 'LDO_USDT', 'STRK_USDT'
]

# 3. Load CoinGecko coin list
coins_list = cg.get_coins_list()
symbol_to_id = {coin['symbol'].upper(): coin['id'] for coin in coins_list}

# 4. Historical MCAP fetcher
def get_historical_mcap(coin_id, date_str):
    try:
        from_ts = int(datetime.strptime(date_str, "%d-%m-%Y").timestamp())
        to_ts = from_ts + 86400
        hist_data = cg.get_coin_market_chart_range_by_id(
            id=coin_id,
            vs_currency='usd',
            from_timestamp=from_ts,
            to_timestamp=to_ts
        )
        if hist_data and "market_caps" in hist_data and hist_data["market_caps"]:
            return hist_data["market_caps"][0][1]
    except Exception as e:
        print(f"❌ Error fetching mcap for {coin_id} on {date_str}: {e}")
    return None

# 5. Iterate + Store Results
token_mcap_on_date = {}

for token in tokens_to_check:
    symbol = token.split('_')[0]
    coin_id = symbol_to_id.get(symbol.upper())

    if coin_id:
        mcap = get_historical_mcap(coin_id, date_str)
        token_mcap_on_date[token] = mcap
        print(f"✅ {token}: Market Cap on {date_str} = ${mcap:,.0f}" if mcap else f"⚠️  {token}: No MCAP found")
        time.sleep(1.2)  # ⏳ Respect rate limits
    else:
        print(f"🚫 {token}: No CoinGecko ID found")

# 6. (Optional) Convert to DataFrame
import pandas as pd

df = pd.DataFrame(list(token_mcap_on_date.items()), columns=["Token", f"MCAP on {date_str}"])
df.sort_values(by=f"MCAP on {date_str}", inplace=True)
df.reset_index(drop=True, inplace=True)
df


✅ PYTH_USDT: Market Cap on 01-08-2024 = $1,324,259,889
✅ CSPR_USDT: Market Cap on 01-08-2024 = $211,878,703
✅ TON_USDT: Market Cap on 01-08-2024 = $7,912,068
✅ NEAR_USDT: Market Cap on 01-08-2024 = $5,698,781,219
⚠️  CELO_USDT: No MCAP found
✅ CRV_USDT: Market Cap on 01-08-2024 = $312,952,045
✅ HBAR_USDT: Market Cap on 01-08-2024 = $2,287,704,617
✅ FET_USDT: Market Cap on 01-08-2024 = $2,987,212,072
✅ FIL_USDT: Market Cap on 01-08-2024 = $2,467,336,884
✅ ADA_USDT: Market Cap on 01-08-2024 = $14,356,487,864
✅ STX_USDT: Market Cap on 01-08-2024 = $332,510
✅ WLD_USDT: Market Cap on 01-08-2024 = $663,398,646
✅ AEVO_USDT: Market Cap on 01-08-2024 = $404,255,857
⚠️  ARB_USDT: No MCAP found
✅ SATS_USDT: Market Cap on 01-08-2024 = $601,219,538
✅ YGG_USDT: Market Cap on 01-08-2024 = $174,737,328
⚠️  SAND_USDT: No MCAP found
✅ MEW_USDT: Market Cap on 01-08-2024 = $635,357,142
❌ Error fetching mcap for warpcore on 01-08-2024: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/

,Token,MCAP on 01-08-2024
0,USDC_USDT,0.000000e+00
1,DOGE_USDT,0.000000e+00
2,SAND_USDT,0.000000e+00
3,XRP_USDT,0.000000e+00
4,WIF_USDT,0.000000e+00
5,ZBCN_USDT,0.000000e+00
6,APE_USDT,0.000000e+00
7,BONK_USDT,0.000000e+00
8,CELO_USDT,0.000000e+00
9,FLOKI_USDT,0.000000e+00


In [9]:
# Assuming your DataFrame is named `df` and looks like the one you shared
min_mcap = 50_000_000
max_mcap = 150_000_000
mcap_column = "MCAP on 01-08-2024"

filtered_df = df[
    (df[mcap_column] >= min_mcap) &
    (df[mcap_column] <= max_mcap)
].copy()

print(f"🎯 Tokens with Market Cap between ${min_mcap:,} and ${max_mcap:,}: {len(filtered_df)}")
display(filtered_df.reset_index(drop=True))


🎯 Tokens with Market Cap between $50,000,000 and $150,000,000: 5


,Token,MCAP on 01-08-2024
0,AIDOGE_USDT,5.778814e+07
1,VELO_USDT,6.413242e+07
2,MERL_USDT,1.004750e+08
3,BIGTIME_USDT,1.021762e+08
4,SUSHI_USDT,1.328246e+08
